#  Vasicek model calibration
**Ref** Chapter 7 of [Hir13]

Our goal is to design pricing engine and calirate vasicek model from data

## Pricing formula for Vasicek model

Notations: 
- $P(t, T)$: zero coupon bond price
- $L(t, T)$: LIBOR rate
- $s(t, T, N)$: swap rate with $N$ terms on $[t, T]$

The relation to the pricing formulas are 
$$L(t, T) = \frac{100}{T-t} ( \frac{1}{P(t, T)} - 1)$$
and 
$$ s(t, T, N) = 100 \frac{1 - P(t, T)}{ \Delta \sum_{j=1}^N P(t, t + j \Delta)}, \hbox{ where } \Delta = \frac{T - t}{N}. $$

If we further assume Vasicek model for the short rates with parameter $\theta = (\kappa, \mu, \sigma, r_0)$, i.e.
$$d r_t = \kappa (\mu - r_t) dt + \sigma dW_t, \hbox{ with } r_0,$$
the rate has its explicit form given by
    $$r_t = r_0 e^{-\kappa t} + \mu ( 1 - e^{-\kappa t}) + \sigma e^{-\kappa t} \int_0^t e^{-\kappa s} dW_s.$$

__Todo__

Verify the above explicit form $r_t$ as the solution to the original vasicek model.

__Todo__

Design pricing engine of ZCB $P(0, T)$ using the above explicit formula.

(Hint) ZCB value is determined by
$$P(0, T) = \mathbb E[e^{- \int_0^T r_t dt}] = e^{A(0, T) - B(0, T) r_0}$$
with 
$$B(t, T) = \frac{1 - e^{-\kappa(T-t)}}{\kappa}, \quad 
A(t, T) = (\mu - \frac{\sigma^2}{2\kappa^2}) [B(t, T) - (T-t)] - \frac{\sigma^2}{4\kappa} B^2(t, T).$$

__Todo__

Design alternative pricing engine of ZCB $P(0,T)$ using exact sampling.

(hint) $R(T) = \int_0^T r_t dt$ follows a normal distribution with
$$\mathbb E R_T = \mu T + (r_0 - \mu) \frac{1 - e^{-\kappa T}}{\kappa}$$ 
and 
$$Var(R_T) = \frac{\sigma^2}{2 \kappa^3} (2 \kappa T - 3 + 4 e^{-\kappa T} - e^{-2\kappa T}).$$

__Todo__

Compute ZCB $P(0,1)$ Libor $L(0,1)$ with the following parameters using above two different pricing engines.

In [1]:
'''======paras======='''
theta = [.1, .05, .003, .03]
kappa, mu, sigma, r0 = theta

__Todo__

find 10 term swap rates with term length 1/2 year, i.e $s(t=0, T =5, N =10)$.

**Todo** 

Pick a date, and using Libor market data of that data, callibrate Vasicek model. 
    - Then compare market rate and calibrated rate in a plot.
    - You may use  SSRE
    - You may do calibration twice using two pricing engines, and see which one is better.
    - You may use the following provided market data too.

In [3]:
import pandas as pd
dfLiborRate = pd.DataFrame({'maturity (months)': [1, 2, 3, 6, 12],
                            '20081029 rate(%)': [3.1175, 3.2738, 3.4200, 3.4275, 3.4213],
                            '20110214 rate(%)': [0.2647, 0.2890, 0.3140, 0.4657, 0.7975]
                          })

In [4]:
dfLiborRate

,20081029 rate(%),20110214 rate(%),maturity (months)
0,3.1175,0.2647,1
1,3.2738,0.2890,2
2,3.4200,0.3140,3
3,3.4275,0.4657,6
4,3.4213,0.7975,12


**Todo** 

Pick a date, and using swap market data of that data, callibrate Vasicek model. 
    - Then compare market rate and calibrated rate in a plot.
    - You may use  SSRE
    - You may do calibration twice using two pricing engines, and see which one is better.
    - You may use the following provided market data too.

In [5]:
dfSwapRate = pd.DataFrame({'term (year)': [2, 3, 5, 7, 10, 15, 30],
                            '20081029 rate(%)': [2.6967, 3.1557, 3.8111, 4.1497, 4.3638, 4.3753, 4.2772],
                            '20110214 rate(%)': [1.0481, 1.5577, 2.5569, 3.1850, 3.7225, 4.1683, 4.4407]
                          })

In [6]:
dfSwapRate

,20081029 rate(%),20110214 rate(%),term (year)
0,2.6967,1.0481,2
1,3.1557,1.5577,3
2,3.8111,2.5569,5
3,4.1497,3.1850,7
4,4.3638,3.7225,10
5,4.3753,4.1683,15
6,4.2772,4.4407,30
